In [47]:
!python data_loading.py

In [48]:
!python model.py

In [49]:
import numpy as np
import os

In [50]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
from model import load_df, preprocess_df, impute_missing

In [52]:
all_data_df = load_df(model_t_plus=1)

In [53]:
all_data_df = impute_missing(all_data_df)

In [54]:
all_data_df, to_pred, gc_time = preprocess_df(all_data_df, months_to_predict=['6.0_2022.0'], model_t_plus=1) 

In [55]:
scope_gcs = ["ESV - PLS",
"ESV - BCM",
"DDV - IT",
"FS - Banking and Capital Markets",
"DDV - Operations",
"Finance - Enterprise Performance Management"]

## Models

In [56]:
from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestRegressor

#model = RandomForestRegressor(n_estimators=125, max_depth = 3)

model_xgb = XGBRegressor(n_estimators = 50, max_depth = 4, tree_method="hist", enable_categorical=True)
#model_xgb = XGBRegressor(n_estimators = 50, max_depth = 4)

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

In [57]:
keep_columns = [ 'Amount added in pipe', 'Entries added in pipe', 'Unique accounts added in pipe', 'Amount won', 'Projects won', 'Unique accounts won', 'EM% mean', 'business_days', 'Fracn of team as As','Fracn of teams as Ms','As underutilized by', 'Ds underutilized by','log of team size']


In [58]:
all_data_df.columns

Index(['Unique accounts added in pipe', 'Amount added in pipe',
       'Entries added in pipe', 'Unique accounts won', 'Amount won',
       'Projects won', 'Senior Associate_Util', 'Associate_Util',
       'Senior Manager_Util', 'Manager_Util', 'Director_Util',
       'Num SMs above target', 'Num SAs above target', 'Num Ds Above Target',
       'Num Ms above target', 'Num As Above Target', 'Num SAs below target',
       'Num SMs below target', 'Num Ms below target', 'Num Ds Below Target',
       'Num As Below Target', 'Ds overutilized by', 'Ms overutilized by',
       'SAs overutilized by', 'As overutilized by', 'SMs overutilized by',
       'SAs underutilized by', 'Ms underutilized by', 'As underutilized by',
       'SMs underutilized by', 'Ds underutilized by', 'Fracn of team as As',
       'Fracn of teams as SMs', 'Fracn of team as SAs', 'Fracn of teams as Ms',
       'Fracn of teams as Ds', 'Revenue Sum', 'EM% mean', 'EM% mean + 1',
       'Revenue Sum + 1', 'Growth Cell', 'busines

In [59]:
y_log = all_data_df['Revenue Sum + 1']

In [60]:
X = all_data_df[keep_columns]

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.30, random_state=42)

In [62]:
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=True,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [63]:
model_xgb.score(X_train, y_train)

0.984259880857257

In [64]:
model_xgb.score(X_test, y_test)

0.8401189093160855

In [65]:
from sklearn.feature_selection import SequentialFeatureSelector

In [66]:
c=['Entries added in pipe', 'Unique accounts added in pipe',
       'Amount added in pipe', 'Projects won', 'Unique accounts won',
       'Amount won', 'Senior Associate_Util', 'Manager_Util', 'Associate_Util',
       'Senior Manager_Util', 'Director_Util', 'Num SMs above target',
       'Num Ds Above Target', 'Num As Above Target', 'Num Ms above target',
       'Num SAs above target', 'Num As Below Target', 'Num Ms below target',
       'Num SAs below target', 'Num Ds Below Target', 'Num SMs below target',
       'SAs overutilized by', 'As overutilized by', 'Ms overutilized by',
       'SMs overutilized by', 'Ds overutilized by', 'Ds underutilized by',
       'SAs underutilized by', 'As underutilized by', 'Ms underutilized by',
       'SMs underutilized by', 'Fracn of team as SAs', 'Fracn of teams as Ms',
       'Fracn of team as As', 'Fracn of teams as SMs', 'Fracn of teams as Ds',
       'Revenue Sum', 'EM% mean', 'EM% mean + 1',
       'business_days', 'log of team size']

In [67]:
from joblib import dump, load
from tqdm import tqdm

In [69]:
for i in tqdm(range(1, len(c))):
    if os.path.exists('features_and_models/' + str(i) + '_sfs_forward.joblib'):
        print('in else')
        pass
    else:
        xgb = XGBRegressor(n_estimators = 50, max_depth = 4, tree_method="hist", enable_categorical=True,random_state=42,n_jobs=-1)
        sfs = SequentialFeatureSelector(xgb,n_features_to_select=i,n_jobs=-1)
        sfs.fit(all_data_df[c], all_data_df['Revenue Sum + 1'])
        dump(sfs, 'features_and_models/' + str(i) + '_sfs_forward.joblib')

#         xgb = XGBRegressor(n_estimators = 50, max_depth = 4, tree_method="hist", enable_categorical=True,random_state=42,n_jobs=-1)
#         sfs = SequentialFeatureSelector(xgb,n_features_to_select=i,direction='backward',n_jobs=-1)
#         sfs.fit(all_data_df[c], all_data_df['Revenue Sum + 1'])
#         dump(sfs, 'features_and_models/' + str(i) + '_sfs_backward.joblib')

100%|███████████████████████████████████████████████████████████████████████████████| 40/40 [47:45<00:00, 71.63s/it]


In [33]:
xgb = XGBRegressor(n_estimators = 50, max_depth = 4, tree_method="hist", enable_categorical=True,random_state=42,n_jobs=-1)
sfs = SequentialFeatureSelector(xgb,n_features_to_select=2,n_jobs=-1)
sfs.fit(all_data_df[c], all_data_df['Revenue Sum + 1'])
dump(sfs, 'features_and_models/' + str(2) + '_sfs_forward.joblib')


['features_and_models/2_sfs_forward.joblib']

In [70]:
stats_forward = []
stats_backward = []

In [71]:
for i in range(1,41):
    sfs = load('features_and_models/' + str(i) + '_sfs_forward.joblib')
    X = sfs.transform(all_data_df[c])
    X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.30, random_state=42)
    if os.path.exists('features_and_models/' + str(i) + '_xgb_forward.joblib'):
        xgb = load('features_and_models/' + str(i) + '_xgb_forward.joblib')
    else:
        xgb = XGBRegressor(n_estimators = 50, max_depth = 4, tree_method="hist", enable_categorical=True)
        xgb.fit(X_train, y_train)
        dump(xgb, 'features_and_models/' + str(i) + '_xgb_forward.joblib')
    stats_forward.append([sfs.get_feature_names_out(), xgb.score(X_train, y_train), xgb.score(X_test, y_test)])
    
#     sfs = load('features_and_models/' + str(i) + '_sfs_backward.joblib')
#     X = sfs.transform(all_data_df[c])
#     X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.30, random_state=42)
#     if os.path.exists('features_and_models/' + str(i) + '_xgb_backward.joblib'):
#         xgb = load('features_and_models/' + str(i) + '_xgb_backward.joblib')
#     else:
#         xgb = XGBRegressor(n_estimators = 50, max_depth = 4, tree_method="hist", enable_categorical=True)
#         xgb.fit(X_train, y_train)
#         dump(xgb, 'features_and_models/' + str(i) + '_xgb_backward.joblib')
#     stats_backward.append([sfs.get_feature_names_out(), xgb.score(X_train, y_train), xgb.score(X_test, y_test)])

In [72]:
stats_forward.sort(key=lambda x: x[2], reverse=True)

In [ ]:
stats_forward[0]

In [73]:
final_features = list(stats_forward[0][0]) + ['Growth Cell']
X = all_data_df[final_features]
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.30, random_state=42)
model_with_growth_cell = XGBRegressor(n_estimators = 50, max_depth = 4, tree_method="hist", enable_categorical=True)
model_with_growth_cell.fit(X_train, y_train)
print(model_with_growth_cell.score(X_train, y_train))
print(model_with_growth_cell.score(X_test, y_test))
print(stats_forward[0][1])
print(stats_forward[0][2])

-16.186520727649427
-4.44402702403491
0.9933924364031038
0.9196920107164297


In [75]:
final_features = list(stats_forward[0][0])
X = all_data_df[final_features]
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.30, random_state=42)
final_model = load('features_and_models/' + str(len(stats_forward[0][0])) + '_xgb.joblib')
print(final_model.score(X_train, y_train))
print(final_model.score(X_test, y_test))

FileNotFoundError: [Errno 2] No such file or directory: 'features_and_models/15_xgb.joblib'

In [ ]:
keep_columns = final_features

In [ ]:
keep_columns # forward selected columns

In [74]:
final_features

['Amount won',
 'Associate_Util',
 'Num SMs above target',
 'Num Ds Above Target',
 'Num As Above Target',
 'Num Ms above target',
 'Num SAs above target',
 'Num As Below Target',
 'Num SAs below target',
 'Num Ds Below Target',
 'As overutilized by',
 'SMs underutilized by',
 'Fracn of team as As',
 'Revenue Sum',
 'EM% mean + 1',
 'Growth Cell']

# Predict for Future

In [ ]:
to_pred_gc_time = to_pred[['Growth Cell', 'Time']]

In [ ]:
#X_copy = to_pred_copy.drop(['Identifier', 'Growth Cell', 'Time'], axis = 1)
X_copy = to_pred[keep_columns]

In [ ]:
# pred_rev = (np.exp(model_xgb.predict(X_copy)) - 1)
pred_rev = (np.exp(final_model.predict(X_copy)) - 1)

In [ ]:
to_pred_gc_time['Revenue Sum'] = pred_rev

In [ ]:
to_pred_gc_time[to_pred_gc_time['Growth Cell'].apply(lambda x: x in scope_gcs)]